# Data manipulations

A matrix transformation is a matrix multiplication between a transformation matrix M and a data matrix D that gives you a manipulated data matrix D' as output.

We can use matrix multiplications to transform our data (our data points, represented as feature vectors).

## But first, some review of dot products

What is being done in this cell?

* Element-wise multiply [4,5,6] and [1,2,3] and then sum
* Element-wise multiply [7,8,9] and [1,2,3] and then sum

In [ ]:
import numpy as np

v = np.array([1,2,3])
m = np.array([[4,5,6], [7,8,9]])

print(m@v)

And in this cell?
* 32: Element-wise multiply [4,5,6] and [1,2,3] and then sum
* 6540: Element-wise multiply [4,5,6] and [10, 100, 1000] and then sum
* 50: Element-wise multiply [7,8,9] and [1,2,3] and then sum
* 9870: Element-wise multiply [7,8,9] and [10, 100, 1000] and then sum

In [ ]:
m2 = np.array([[1,2,3], [10, 100, 1000]])

print(m@m2.T)

## Load and look at our data

Let's load the used car data.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

data = np.array(np.genfromtxt('data/vehiclesNumeric.csv', delimiter=',', skip_header=1, dtype=int, encoding="utf-8"))  

# get a pandas dataframe for plotting
df = pd.DataFrame(data, columns=["id", "price", "year", "odometer"])

Let's get some **summary statistics**.

In [ ]:
def getSummaryStatistics(data):
    print("min, max, mean, std per variable")
    return pd.DataFrame([data.min(axis=0), data.max(axis=0), data.mean(axis=0), data.std(axis=0)])

def getShapeType(data):
    print("shape")
    return (data.shape, data.dtype)

print(getSummaryStatistics(data))
print(getShapeType(data))

Let's **reduce the data** to two dimensions, just year and price.


In [ ]:
# How are we going to get just those two columns?
reducedData = data[:, [1,2]]

print(getSummaryStatistics(reducedData))
print(getShapeType(reducedData))

Let's plot the used car data.

In [ ]:
def plot2d(data):
    sns.scatterplot(pd.DataFrame(data[:, [0, 1]], columns=["price", "year"]), x="year", y="price").set(title="Year vs price for Craigslist used car listings")
    
plot2d(reducedData)

## Translation

Translation is a kind of data transformation where we move data around, but each data point stays the same distance away from every other data point.

Translation is a two step process:
* Add homogeneous coordinate
* Do translation as matrix multiplication

### Add homogeneous coordinate

We need to add a dummy column of ones so we can do the matrix multiplication. Why? See https://www.sciencedirect.com/topics/mathematics/homogeneous-coordinate.

In [ ]:
# How do we append a whole column?
homogenizedData = np.append(reducedData, np.array([np.ones(reducedData.shape[0], dtype=int)]).T, axis=1)
print("homogenized data")
print(getSummaryStatistics(homogenizedData))
print(getShapeType(homogenizedData))

### Translate

Let's **translate** that year column so that it too starts at 0.

A translation matrix for two-variable data looks like:
$$\begin{pmatrix} 1 & 0 & x \\ 0 & 1 & y \\ 0 & 0 & 1\end{pmatrix}$$
where $x, y$ are the amount by which you want the $0th$ and $1st$ variables translated, respectively.



In [ ]:
# we need to define a transformation matrix that will allow us to shift the price variable; this one will be the identity matrix with the translation specified in an extra last column
translateTransform = np.eye(homogenizedData.shape[1], dtype=int)
translateTransform[1, 2] = -reducedData[:, 1].min()
print("transformMatrix")
print(translateTransform)

print(homogenizedData[0:4])

# now we need to do the translation
transformedData = (translateTransform@homogenizedData.T).T
print("after translation, transformedData")
print(getSummaryStatistics(transformedData))
print(getShapeType(transformedData))
plot2d(transformedData)

Check:
* only the summary statistics for year should have changed
* the standard deviation for year should be the same

## Scaling

Scaling is kind of data transformation where we increase or decrease the range of one or more variables.

### Scaling on its own

Let's **scale** that year column so it's months instead of years.

A scaling matrix for two-variable data looks like:
$$\begin{pmatrix} x & 0 \\ 0 & y \end{pmatrix}$$
where $x, y$ are the amount by which you want the $0th$ and $1st$ variables scaled, respectively. 


In [ ]:
scaleTransform = np.eye(reducedData.shape[1], dtype=float)
scaleTransform[1, 1] = 12
print("transformMatrix")
print(scaleTransform)

transformedData = (scaleTransform@reducedData.T).T
print("after scaling, transformedData")
print(getSummaryStatistics(transformedData))
print(getShapeType(transformedData))
plot2d(transformedData)

Check:
* only the summary statistics for year should have changed

### Scaling together with other transformations

If you want to translate *and* scale, you just add the homogeneous coordinate into the scaling matrix too!
$$\begin{pmatrix} x & 0 & 0\\ 0 & y & 0 \\ 0 & 0 & 1 \end{pmatrix}$$



In [ ]:
scaleTransform = np.eye(homogenizedData.shape[1], dtype=float)
scaleTransform[1, 1] = 12
print("transformMatrix")
print(scaleTransform)

translateTransform = np.eye(homogenizedData.shape[1], dtype=float)
translateTransform[1, 2] = -reducedData[:, 1].min()
print("transformMatrix")
print(translateTransform)

transformMatrix = translateTransform@scaleTransform

transformedData = (transformMatrix@homogenizedData.T).T
print("after scaling, transformedData")
print(getSummaryStatistics(transformedData))
print(getShapeType(transformedData))
plot2d(transformedData)

Check:
* Although we added the homogeneous coordinate, the scaling produced the same output as before

## Global max-min normalization

Let's try **global (max-min) normalization**

Here is how that works:
1. subtract the global minimum from each datapoint
2. divide by the global range (max - min)

The transformation matrix for two variables looks like:
$$\begin{pmatrix} \frac{1}{(max(data)-min(data))} & 0 & -min(data) \\ 0 & \frac{1}{(max(data)-min(data))} & -min(data) \\ 0 & 0 & 1\end{pmatrix}$$

What does this look like in terms of scaling and translation?

In [ ]:
# subtract the global minimum from each datapoint
translateTransform = np.eye(homogenizedData.shape[1], dtype=float)
for i in range(reducedData.shape[1]):
    translateTransform[i, 2] = -reducedData.min()
print("translateTransform")
print(translateTransform)

# divide by the global range
scaleTransform = np.eye(3, 3)
scaleTransform[0, 0] = 1 / (reducedData.max() - reducedData.min())
scaleTransform[1, 1] = 1 / (reducedData.max() - reducedData.min())
scaleTransform[2, 2] = 1 / (reducedData.max() - reducedData.min())
print("scaleTransform")
print(scaleTransform)

# when we do a series of transformations, first we multiply the smaller transformation matrices, and only at the end the result of that with the larger data matrix (more efficient!)
totalTransform = scaleTransform@translateTransform
print("transformMatrix")
print("shape\n", getShapeType(totalTransform))
print(totalTransform)


In [ ]:
transformedData = (totalTransform@homogenizedData.T).T
print("after global normalization, transformedData")
print(getSummaryStatistics(transformedData))
print(getShapeType(transformedData))
plot2d(transformedData)

What seems weird to you about this process? What seems solid?

## Per-variable max-min normalization

Let's try **max-min normalization per variable**.

The transformation matrix for two variables, $x$ and $y$, looks like:
$$\begin{pmatrix} \frac{1}{(max(x)-min(x))} & 0 & -min(x) \\ 0 & \frac{1}{(max(y)-min(y))} & -min(y) \\ 0 & 0 & 1\end{pmatrix}$$

What does this look like in terms of scaling and translation?

In [ ]:
print("before per variable normalization, homogenizedData")
print(getSummaryStatistics(homogenizedData))

translateTransform = np.eye(homogenizedData.shape[1], dtype=float)
translateTransform[:, 2] = np.array([-homogenizedData[:, 0].min(), -homogenizedData[:, 1].min(), 1])
scaleTransform = np.eye(homogenizedData.shape[1]) * [1 / (homogenizedData[:, 0].max() - homogenizedData[:, 0].min()), 1 / (homogenizedData[:, 1].max() - homogenizedData[:, 1].min()), 1]

totalTransform = scaleTransform@translateTransform
print("transformMatrix")
print(totalTransform)


transformedData = (totalTransform @ homogenizedData.T).T
print("after per variable normalization, transformedData")
print(getSummaryStatistics(transformedData))
print(getShapeType(transformedData))
plot2d(transformedData)

What is good about this way of normalizing our data? What do we not like?

## Z-scoring

Max-min normalization will move everything to the unit square, but that may not help me see things more clearly. What if I try **z-scoring**: normalizing each feature by its mean and standard deviation instead?

The transformation matrix for two variables, $x$ and $y$, looks like:
$$\begin{pmatrix} \frac{1}{(std(x))} & 0 & -mean(x) \\ 0 & \frac{1}{(std(y))} & -mean(y) \\ 0 & 0 & 1\end{pmatrix}$$


In [ ]:
print("before z-scoring, homogenizedData")
print(getSummaryStatistics(homogenizedData))

translateTransform = np.eye(homogenizedData.shape[1], dtype=float)
translateTransform[:, 2] = np.array([-homogenizedData[:, 0].mean(), -homogenizedData[:, 1].mean(), 1], dtype=float)
print(translateTransform)
scaleTransform = np.eye(homogenizedData.shape[1]) * [1 / homogenizedData[:, 0].std(), 1 / homogenizedData[:, 1].std(), 1]

totalTransform = scaleTransform@translateTransform
print("transformMatrix")
print(totalTransform)

transformedData = (totalTransform@homogenizedData.T).T
print("after z-scoring, transformedData")
print(getSummaryStatistics(transformedData))
print(getShapeType(transformedData))
plot2d(transformedData)

## Challenges

* What if I want to *mean center* my data, or make the mean point zero?
* What if I want to *mean center* and normalize so that the data's range is [-1, 1]?


## Visualizations

There are some nice web-based visualizers that show you how scaling and rotation work (rotation comes next week!). Here is one where you can upload your own picture:
* https://web.ma.utexas.edu/users/ysulyma/matrix/

And here is one that walks through the matrix multiply with you:
* https://www.cs.usfca.edu/~galles/visualization/RotateScale3D.html
* https://www.cs.usfca.edu/~galles/visualization/RotateScale2D.html

And finally, here is an Observable notebook that also reviews the matrix multiply:
* https://observablehq.com/@noonat/transformation-matrices

Happy playing!

## Resources

* https://staff.fnwi.uva.nl/r.vandenboomgaard/IPCV20162017/LectureNotes/MATH/homogenous.html
* https://www.informit.com/articles/article.aspx?p=2854376&seqNum=8
* https://towardsdatascience.com/normalization-techniques-in-python-using-numpy-b998aa81d754
* https://www.machinecurve.com/index.php/2020/11/19/how-to-normalize-or-standardize-a-dataset-in-python/